In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pymysql

In [18]:
# 1. '.env'파일을 읽어와서, 키=값으로 이뤄진 딕셔너리 생성
envs = {}

with open('.env', 'r') as f:
    for l in f.readlines():
        k, v = l.rstrip().split('=')
        envs[k] = v
        
envs    

{'DB_HOST': 'localhost',
 'DB_PORT': '3306',
 'DB_USER': 'jeon99yu',
 'DB_PASSWORD': 'test1234',
 'DB_DATABASE': 'study',
 'GPT_API_KEY': 'asfdagfdshmgpfdskfpasnfodfsapki'}

In [19]:
# 2. 접속정보 envs 활용해서 넣기 (port는 정수형으로)
conn = pymysql.connect(
    host=envs['DB_HOST'], # 19.222.124.233
    port=int(envs['DB_PORT']),
    user=envs['DB_USER'], 
    password=envs['DB_PASSWORD'], 
    database=envs['DB_DATABASE'], 
    charset='utf8mb4'
)

In [ ]:
# 3. envs를 활용하여 db 정보만 담고있는 db_config를 만든 뒤, 다음의 코드가 실행되도록 하기
db_config = {} 
# db_config = {
#     'host': envs['DB_HOST'],
#     'port': int(envs['DB_PORT']),
#     'user': envs['DB_USER'],
#     'password': envs['DB_PASSWORD'],
#     'database': envs['DB_DATABASE']
# }

for k, v in envs.items():
    print(k,v)

    if 'DB' not in k: # 키에 db가 없으면 넘어가고
        continue
    # _로 분리하여 connect에 필요한 인자이름으로 바꿔줌 (host, port, user, password, database)
    k = k.split('_')[1].lower()
    print(k,v)
    # port 인 경우, v에 대해 int 처리
    if k == 'port':
        v = int(v)
        
    db_config[k] = v

conn = pymysql.connect(**db_config)
db_config

DB_HOST localhost
host localhost
DB_PORT 3306
port 3306
DB_USER jeon99yu
user jeon99yu
DB_PASSWORD test1234
password test1234
DB_DATABASE study
database study
GPT_API_KEY asfdagfdshmgpfdskfpasnfodfsapki


{'host': 'localhost',
 'port': 3306,
 'user': 'jeon99yu',
 'password': 'test1234',
 'database': 'study'}

In [23]:
envs

{'DB_HOST': 'localhost',
 'DB_PORT': '3306',
 'DB_USER': 'jeon99yu',
 'DB_PASSWORD': 'test1234',
 'DB_DATABASE': 'study',
 'GPT_API_KEY': 'asfdagfdshmgpfdskfpasnfodfsapki'}

In [ ]:
with conn.cursor() as cur:
    result = cur.execute('INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
                         ('david', 34, '경기') #Value
                         )
conn.commit()

In [28]:
# 한 SQL문을 여러번 실행하고 싶을 때 executemany
with conn.cursor() as cur:
    result = cur.executemany('INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
                         [('david', 34, '경기'),('Joe', 22, '서울'),('zero', 17, '서울')] #Value
                         )
conn.commit()

In [32]:
import pymysql.cursors

with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur:
    result = cur.execute('SELECT * FROM Students')
    datas = cur.fetchall() # fetchmany(size), fetchone()
result

9

In [33]:
datas

[{'StudentID': 1, 'Name': '홍길동', 'Age': 30, 'Address': '인천'},
 {'StudentID': 2, 'Name': '이연걸', 'Age': 60, 'Address': '서울'},
 {'StudentID': 3, 'Name': '이몽룡', 'Age': 42, 'Address': '대전'},
 {'StudentID': 4, 'Name': '성춘향', 'Age': 27, 'Address': '경기'},
 {'StudentID': 5, 'Name': 'Jason', 'Age': 55, 'Address': '서울'},
 {'StudentID': 6, 'Name': 'david', 'Age': 34, 'Address': '경기'},
 {'StudentID': 7, 'Name': 'david', 'Age': 34, 'Address': '경기'},
 {'StudentID': 8, 'Name': 'Joe', 'Age': 22, 'Address': '서울'},
 {'StudentID': 9, 'Name': 'zero', 'Age': 17, 'Address': '서울'}]

In [34]:
def insert_student(name, age, address):
    with conn.cursor() as cur:
        result = cur.execute(
            'INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
            (name, age, address))
    conn.commit()
insert_student('이름', 22, '')

In [35]:
conn.close()